In [4]:
!pip install git+https://github.com/GTSF-Quantitative-Sector/sec_parser.git

  Cloning https://github.com/GTSF-Quantitative-Sector/sec_parser.git to /tmp/pip-req-build-7zevqo7c
  Running command git clone --filter=blob:none --quiet https://github.com/GTSF-Quantitative-Sector/sec_parser.git /tmp/pip-req-build-7zevqo7c
  Resolved https://github.com/GTSF-Quantitative-Sector/sec_parser.git to commit 3bde3fd9b289cbcf2a38fac9790c0fbf2f47b962
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 21.9 MB/s eta 0:00:00
  Created wheel for sec: filename=sec-1.5-py3-none-any.whl size=20610135 sha256=73adaad4dda4ccd49c7c6312b7db6a

In [5]:
from sec import constants, stock, lookups, processor
import pandas as pd
from datetime import datetime, timedelta
from typing import Iterator
import asyncio
from tqdm import tqdm
import pickle
import os
import matplotlib.pyplot as plt

In [6]:
constants.set_polygon_key("_4BtZn3PRCLu6fsdu7dgddb4ucmB1sfp")

In [7]:
API_KEY = "_4BtZn3PRCLu6fsdu7dgddb4ucmB1sfp"

In [9]:
# Get the current date
def get_date(day: int = 0):
# Format the date as YYYY-MM-DD
  today = datetime.now()
  day = today - timedelta(days=day)
  date = day.strftime('%Y-%m-%d')
  return date

In [12]:
# Hard coded every industry's Beta
industry_beta = {
  "Advertising": 1.37,
  "Aerospace/Defense": 1.08,
  "Air Transport": 1.27,
  "Apparel": 1.19,
  "Auto & Truck": 1.52,
  "Auto Parts": 1.34,
  "Bank (Money Center)": 1.06,
  "Banks (Regional)": 0.46,
  "Beverage (Alcoholic)": 1.13,
  "Beverage (Soft)": 0.76,
  "Broadcasting": 1.06,
  "Brokerage & Investment Banking": 1.12,
  "Building Materials": 1.32,
  "Business & Consumer Services": 1.02,
  "Cable TV": 1.28,
  "Chemical (Basic)": 1.1,
  "Chemical (Diversified)": 1.13,
  "Chemical (Specialty)": 1.09,
  "Coal & Related Energy": 1.27,
  "Computer Services": 1.0,
  "Computers/Peripherals": 1.13,
  "Construction Supplies": 0.94,
  "Diversified": 1.19,
  "Drugs (Biotechnology)": 1.12,
  "Drugs (Pharmaceutical)": 1.03,
  "Education": 1.23,
  "Electrical Equipment": 1.24,
  "Electronics (Consumer & Office)": 1.3,
  "Electronics (General)": 0.93,
  "Engineering/Construction": 1.06,
  "Entertainment": 0.99,
  "Environmental & Waste Services": 0.91,
  "Farming/Agriculture": 0.99,
  "Financial Svcs. (Non-bank & Insuran": 1.14,
  "Food Processing": 0.71,
  "Food Wholesalers": 0.97,
  "Furn/Home Furnishings": 1.11,
  "Green & Renewable Energy": 1.11,
  "Healthcare Products": 1.06,
  "Healthcare Support Services": 1.03,
  "Heathcare Information and Technol": 1.27,
  "Homebuilding": 1.37,
  "Hospitals/Healthcare Facilities": 0.88,
  "Hotel/Gaming": 1.34,
  "Household Products": 0.78,
  "Information Services": 0.93,
  "Insurance (General)": 1.03,
  "Insurance (Life)": 0.77,
  "Insurance (Prop/Cas.)": 0.74,
  "Investments & Asset Management": 0.46,
  "Machinery": 1.03,
  "Metals & Mining": 0.96,
  "Office Equipment & Services": 1.14,
  "Oil/Gas (Integrated)": 0.67,
  "Oil/Gas (Production and Exploratio": 0.93,
  "Oil/Gas Distribution": 0.79,
  "Oilfield Svcs/Equip.": 0.98,
  "Packaging & Container": 0.87,
  "Paper/Forest Products": 1.94,
  "Power": 0.65,
  "Precious Metals": 0.87,
  "Publishing & Newspapers": 0.92,
  "R.E.I.T.": 1.02,
  "Real Estate (Development)": 0.67,
  "Real Estate (General/Diversified)": 0.56,
  "Real Estate (Operations & Services)": 1.08,
  "Recreation": 1.17,
  "Reinsurance": 0.66,
  "Restaurant/Dining": 1.19,
  "Retail (Automotive)": 1.49,
  "Retail (Building Supply)": 1.94,
  "Retail (Distributors)": 1.11,
  "Retail (General)": 1.25,
  "Retail (Grocery and Food)": 0.49,
  "Retail (REITs)": 1.12,
  "Retail (Special Lines)": 1.18,
  "Rubber& Tires": 0.67,
  "Semiconductor": 1.5,
  "Semiconductor Equip": 1.53,
  "Shipbuilding & Marine": 0.81,
  "Shoe": 1.29,
  "Software (Entertainment)": 1.11,
  "Software (Internet)": 1.62,
  "Software (System & Application)": 1.29,
  "Steel": 1.13,
  "Telecom (Wireless)": 1.09,
  "Telecom. Equipment": 1.08,
  "Telecom. Services": 0.78,
  "Tobacco": 1.22,
  "Transportation": 1.0,
  "Transportation (Railroads)": 1.02,
  "Trucking": 1.15,
  "Utility (General)": 0.58,
  "Utility (Water)": 0.71,
  "Total Market": 1.0,
  "Total Market (without financials)": 1.1
}

In [23]:
# Confining industry_beta dictionary to all only industries with Betas greater than 1.2
keys = list(industry_beta.keys())
for key in keys:
  if (abs(industry_beta[key]) < 1.19):
    del industry_beta[key]
for industry in industry_beta:
  print("Industry: " + str(industry) + " Beta: " + str(industry_beta[industry]))


Industry : Advertising Beta: 1.37
Industry : Air Transport Beta: 1.27
Industry : Apparel Beta: 1.19
Industry : Auto & Truck Beta: 1.52
Industry : Auto Parts Beta: 1.34
Industry : Building Materials Beta: 1.32
Industry : Cable TV Beta: 1.28
Industry : Coal & Related Energy Beta: 1.27
Industry : Diversified Beta: 1.19
Industry : Education Beta: 1.23
Industry : Electrical Equipment Beta: 1.24
Industry : Electronics (Consumer & Office) Beta: 1.3
Industry : Heathcare Information and Technol Beta: 1.27
Industry : Homebuilding Beta: 1.37
Industry : Hotel/Gaming Beta: 1.34
Industry : Paper/Forest Products Beta: 1.94
Industry : Restaurant/Dining Beta: 1.19
Industry : Retail (Automotive) Beta: 1.49
Industry : Retail (Building Supply) Beta: 1.94
Industry : Retail (General) Beta: 1.25
Industry : Semiconductor Beta: 1.5
Industry : Semiconductor Equip Beta: 1.53
Industry : Shoe Beta: 1.29
Industry : Software (Internet) Beta: 1.62
Industry : Software (System & Application) Beta: 1.29
Industry : Tobac

In [26]:
## Finds stocks with a realized volatility that is greater than their Industry's Beta from the past day
async def find_stocks():
  # Gets all S&P500 stocks
  curr_sp = lookups.get_sp500_tickers()

  # Gets the YYYY-MM-DD value of Today and Yesterday
  today = get_date(1)
  yesterday = get_date(2)

  # Gets the percent change of the price of SPY from yesterday to today
  spy = stock.Stock(str("SPY"))
  todays_price = await spy.get_price(today)
  yesterdays_price = await spy.get_price(yesterday)
  spy_price_change = (abs((todays_price - yesterdays_price)/yesterdays_price)) * 100

  # Stores the stocks we are going to purchase options form and their prices.
  buy_stocks = {}

  # Keeps track of number of stocks added to buy_stocks. Limiting to 5**
  i = 0

  # Runs through every stock in the S&P 500
  for ticker in curr_sp:
    try:
      # Allows for us to get the price and industry of ticker
      tick = stock.Stock(str(ticker))
      industry = lookups.get_industry(ticker)

      # We want the industry beta to be greater than 1 to ensure that it is volatile in order to make gamma scalping possible
      if (industry not in industry_beta):
          continue

      try:
      # Tries to get the price of the stock from today and yesterday
        todays_price = await tick.get_price(today)
        yesterdays_price = await tick.get_price(yesterday)
      except:
      # Skips stock if price is not available
        continue

      # Gets the percent change of the price from yesterday to today
      price_change = (abs((todays_price - yesterdays_price)/yesterdays_price)) * 100

      # 1. Divides the percent change in price of the stock divided by the industry beta. We should do this because if the beta is 1.5 than
      # for every percent change in SPY the respective stock should have changed by 1.5. This will allow us to see if the stock actually increased
      # more than 1.5% per SPY's 1%.
      # 2. Multiplies Spy's percent change in price by 15 to find the stocks that have a huge difference in percent increase from their industry Beta.
      # 3. Only accounting for 5 stocks currently, but this could be changed. I just wanted simplicity
      # 4. Goal for future: Want to use Implied Volatility of a stock instead of Beta so we can compare Realized vs Implied volatility from a current day.

      if (price_change/industry_beta[industry] > spy_price_change * 15 and i < 5):
        buy_stocks[ticker] = todays_price
        print(ticker + " Stock Price: " + str(todays_price))
        i += 1
        # Once 5 stocks are found return the stock options we want to buy
        if (i == 5):
          return buy_stocks
    except:
      #Stock not found in SEC Parser
      continue;

await find_stocks()

ADI Price: 193.72
CHTR Price: 287.3
FSLR Price: 150.03
FTNT Price: 65.34
GRMN Price: 133.58


{'ADI': 193.72, 'CHTR': 287.3, 'FSLR': 150.03, 'FTNT': 65.34, 'GRMN': 133.58}

In [12]:
# Next Steps in Creating Gamma Scalping Strategy:
# **Note: Using Polygon API for the options is going to be how we complete most of this work, so make sure you look inot their API docs and use all the correct end points.**
#     1. Determine how much stock we need to buy to become delta nuetral when buying options
#           - For every call or put option you plan to buy, you consider it as buying 10 of those options.
#           - To hedge a long call position, you buy puts. To hedge a long put position, you buy stocks.
#           - Your position's delta adjustment is calculated as: 10×Delta×100, where 10 is the multiplier for
#           the number of options, Delta is the option's delta, and 100 represents the standard number of
#           shares per option contract.
#           - Data Required:
#             - Delta --> For Calls: Delta ranges from 0 to 1. For Puts: Delta ranges from -1 to 0.
#             - Stock Price --> The current price of the underlying stock. This is crucial for calculating
#               the financial implications of buying or selling the stock to hedge your options. This will
#               be provided by the find_stocks fuction which will return a dictionary of stocks and their prices.

#     2. Determine when an option is above/below the strike price for calls/puts respectively
#.        - At the end of each day, look at closing price and see if it is above or below strike price respectively
#         - Data Required:
#           - Strike Price
#     3. If call/put is above/below the strike price we need to also buy stock or puts
#         - In order to remain delta nuetral we need to recalculate our exposure to delta and see how many stocks/puts we need to buy
#.          when we are in the money(above/below strike price).
#         - Data Required:
#.          - new Delta --> For Calls: Delta ranges from 0 to 1. For Puts: Delta ranges from -1 to 0.
#           - New Stock Price for puts/ New Put price for calls --> For hedging
#     4. If call/put goes below/above strike price respectively we need to now sell our stocks or puts
#         - In order to remain delta nuetral we need to recalculate our exposure to delta and see how many stocks/puts we need to sell
#.          when we become out of the money(below/above strike price).
#         - Data Required:
#.          - new Delta --> For Calls: Delta ranges from 0 to 1. For Puts: Delta ranges from -1 to 0.
#           - New Stock Price for puts/ New Put price for calls --> For hedging
#     5. Once there is a day before expiration we do not want ot dno any more scalping sicne there is a lot of weird things that go on with options on expirary


# Back Test:
# When creating the back test, we will use these algorithms. Obviously we do not have the ability to asynchronously make these trades. So the back test will allow us to
# pretend that we are making these trades "asynchronously". So for right now our objective is to make sure our logic works so we can fine tune it once we start testing it out.
#
#